In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, row_number, udf, dayofweek, col, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.types import TimestampType


In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
config['AWS']['AWS_SECRET_ACCESS_KEY']

'EUjXxhztlHSDLQw+PLwU9FzcHlap4kHsI1ewBT/y'

In [4]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
        .getOrCreate()
# "org.apache.hadoop:hadoop-aws:2.7.0"

In [5]:
input_data = "s3a://udacity-dend/"
song_data = os.path.join(input_data + 'song_data/*/*/*/*.json')

songs_df = spark.read.json(song_data)



In [6]:
songs_df

DataFrame[artist_id: string, artist_latitude: double, artist_location: string, artist_longitude: double, artist_name: string, duration: double, num_songs: bigint, song_id: string, title: string, year: bigint]

In [7]:
songs_df.head()

Row(artist_id='AR4T2IF1187B9ADBB7', artist_latitude=63.96027, artist_location='<a href="http://billyidol.net" onmousedown=\'UntrustedLink.bootstrap($(this), "fc44f8f60d13ab68c56b3c6709c6d670", event)\' target="_blank" rel="nofollow">http://billyidol.net</a>', artist_longitude=10.22442, artist_name='Billy Idol', duration=233.22077, num_songs=1, song_id='SOVIYJY12AF72A4B00', title='The Dead Next Door (Digitally Remastered 99)', year=1983)

In [8]:
songs_table = songs_df.select(col("song_id"), col("title"), col("artist_id"), col("year"),col("duration")).distinct()

In [10]:
songs_table.head()

Row(song_id='SOVIYJY12AF72A4B00', title='The Dead Next Door (Digitally Remastered 99)', artist_id='AR4T2IF1187B9ADBB7', year=1983, duration=233.22077)

In [12]:
output_data = "s3a://sparkifyout/output/"
songs_table.write.partitionBy("year", "artist_id").parquet(output_data + 'songs/').mode("overwrite")

Py4JJavaError: An error occurred while calling o63.parquet.
: ExitCodeException exitCode=-1073741515: 
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.Ra

In [ ]:
songs_table*